In [5]:
#game_1 : "find_this_mii"

import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 4820
#####################
#a
#######################
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq);
ret, img = cap.read()
out_size = (480, 270)
img = cv2.resize(img, out_size, 0, 0, interpolation=cv2.INTER_AREA)
height, width = img.shape[0:2]
print(img.shape)
show_img = np.copy(img)
mouse_pressed = False
y0 = x0 = w = h = 0
########################
#b 自行手動抓取模板 選取後按'a'確定
########################
def mouse_callback(event, _x, _y, flags, param):
    global show_img, x0, y0, w, h, mouse_pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_pressed = True
        x0, y0 = _x, _y
        show_img = np.copy(img)
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_pressed:
            show_img = np.copy(img)
            cv2.rectangle(show_img, (x0, y0), (_x, _y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_pressed = False
        w, h = _x - x0, _y - y0
cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)

#Acquire a template through mouse dragging
while True:
    cv2.imshow('image', show_img)
    k = cv2.waitKey(1)
    if k == ord('a') and not mouse_pressed:
        if w*h > 0:
            break
cv2.destroyAllWindows()

template = np.copy(img[y0:y0+h, x0:x0+w])
print(template.shape)
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
show_img = np.copy(img)

cv2.imshow('image', template)
###################################
while True:
    frame_seq += 1
    if frame_seq > 5000:
        frame_seq = 4820
    if frame_seq < 4880:
        frame_seq = 4880
    
    
    
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, img = cap.read()
    if not status_cap:
        break
    img = cv2.resize(img, out_size, 0, 0, interpolation=cv2.INTER_AREA)
    
    k = cv2.waitKey(1)
    if k == 27:
        break
    elif k > 0 and chr(k).isdigit():
        index = int(chr(k))
        if 0 <= index < len(methods):
            method = methods[index]
            res = cv2.matchTemplate(img, template, eval(method))
            print('min:{} max:{}'.format(np.min(res), np.max(res)))
            res = cv2.normalize(res, None, 0, 1, cv2.NORM_MINMAX)            
            print('min_n:{} max_n:{}'.format(np.min(res), np.max(res)))
            #######
            min_max = cv2.minMaxLoc(res)
            if index == 4 or index == 5:   #根据不同的模式最佳匹配位置取值方法不同
                match_loc = min_max[2]
            else:
                match_loc = min_max[3]  
            right_bottom = (match_loc[0] + template.shape[1], match_loc[1] + template.shape[0])
            print('result.min_max:',min_max)
            print('match_loc:',match_loc)
            print('right_bottom',right_bottom)
            ########
            # min_max = cv2.minMaxLoc(res)
            #
            #if index >= methods.index('cv2.TM_SQDIFF'):
            #    loc = np.where(res < 0.2)
            #else:
            #    loc = np.where(res > 0.8)            
            #for pt in zip(*loc[::-1]):
            #    cv2.rectangle(res, (pt[0] - int(w/2), pt[1] - int(h/2)), (pt[0] + int(w/2), pt[1] + int(h/2)),(0, 0, 0), 1)
            res_img = np.copy(res)
            res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR)
            for y in range(0, res.shape[0]):
                for x in range(0, res.shape[1]):
                    if (index >= methods.index('cv2.TM_SQDIFF')) and (res[y,x] < 0.2):            
                        cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 1)
                    elif (index < methods.index('cv2.TM_SQDIFF')) and (res[y,x] > 0.8):            
                        cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 1)
            #######     
            #######
            #print(res_img.shape)
            res_img = cv2.resize(res_img, (width, height))*255
            res_img = res_img.astype(np.uint8)
            #print(res_img.shape)
            cv2.putText(res_img, method, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
            cv2.rectangle(res_img, match_loc,right_bottom, (0,0,255), 5, 8, 0 ) 
            show_img = np.copy(img)
            show_img = cv2.resize(show_img, (width, height))
            print(show_img.shape)
            cv2.rectangle(show_img, match_loc,right_bottom, (0,0,255), 5, 8, 0 ) 

            show_img = np.hstack((show_img, res_img)) 
    ###
    cv2.imshow("now_img", img)
    cv2.imshow('find_this_mii', show_img)
        
cap.release()
cv2.destroyAllWindows()

(270, 480, 3)
(19, 19, 3)
min:0.17087361216545105 max:1.0
min_n:-4.8918238348960585e-09 max_n:1.0
result.min_max: (-4.8918238348960585e-09, 1.0, (89, 212), (109, 95))
match_loc: (89, 212)
right_bottom (108, 231)
(270, 480, 3)
min:0.17226606607437134 max:1.0
min_n:-6.086608550504025e-09 max_n:1.0
result.min_max: (-6.086608550504025e-09, 1.0, (89, 215), (108, 98))
match_loc: (89, 215)
right_bottom (108, 234)
(270, 480, 3)
min:0.1745903491973877 max:1.0
min_n:1.0788596682687057e-09 max_n:1.0
result.min_max: (1.0788596682687057e-09, 1.0, (88, 219), (108, 101))
match_loc: (88, 219)
right_bottom (107, 238)
(270, 480, 3)
min:0.18106190860271454 max:1.0
min_n:5.58405766071246e-09 max_n:1.0
result.min_max: (5.58405766071246e-09, 1.0, (88, 222), (108, 104))
match_loc: (88, 222)
right_bottom (107, 241)
(270, 480, 3)
min:0.17372898757457733 max:1.0
min_n:4.512491713626332e-09 max_n:1.0
result.min_max: (4.512491713626332e-09, 1.0, (86, 224), (108, 107))
match_loc: (86, 224)
right_bottom (105, 243)


b) 一開始需要自行手動抓取模板 選取後按'a'確定  
要看到效果需要持續的案0-5，通過0-5來選取需要的方法分別是  
['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']  
六種方法  
c)找到跟模板最像的臉以後就用紅框框起來  

  

原本不用minMaxLoc()的方法時，經過測試'cv2.TM_CCOEFF'的效果比較好  
不會找到太多奇怪的特徵，抓到的特徵比較少，但可以比較清楚的看到有抓到(藍色框框)  
使用minMaxLoc()時，只有在方法'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'有較好的效果(紅色框框)  
多數的情況可以找到對的特徵點，'cv2.TM_SQDIFF'表現又比較好一點點  
看起來要使用這兩個方法時，是一定要用到minMaxLoc()，不然基本看不出效果來  

In [6]:
#game_2 : "find_two_look_alike"

import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
start_seq = 2180
frame_seq = start_seq


cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
status_cap, frame = cap.read()

###
#找路人
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
height, width =frame.shape[0:2]
locations, weights = hog.detectMultiScale(frame, hitThreshold = 0.15, winStride=(1,1), groupThreshold = 1)


while True:
    frame_seq += 1
    if frame_seq > 2380:
        frame_seq = start_seq
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame = cap.read()
    if not status_cap:
        break
        
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor)
    
    #負責找所有的臉
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=2)
    #負責找所有路人
    locations, _ = hog.detectMultiScale(frame)
    frame_out = np.copy(frame)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(frame_out, (x,y), (x+w,y+h), (255,0,0), 3)

    ####
    #
    if len(face_rects) >= 2:
        for (x0, y0, w0, h0) in face_rects:
            for (x1, y1, w1, h1) in face_rects:
                #if x0 == x1 and y0 == y1 and w0 == w1 and h0 == h1:
                face0 = frame[y0+int(h0/4):y0+int(h0*3/4), x0+int(w0/4):x0+int(w0*3/4)]
                face1 = frame[y1+int(h1/4):y1+int(h1*3/4), x1+int(w1/4):x1+int(w1*3/4)]
                gray0 = cv2.cvtColor(face0, cv2.COLOR_BGR2GRAY)
                gray1 = cv2.cvtColor(face1, cv2.COLOR_BGR2GRAY)
                wt,ht = face0.shape[0:2]
                gray1 = cv2.resize(gray1, (wt,ht))
                #計算相似程度
                diff = cv2.absdiff(gray0, gray1)
                #print("diff= " + str(cv2.mean(diff)[0]))
                #很像時，用紅色框起來
                if cv2.mean(diff)[0] < 30 and cv2.mean(diff)[0] > 1:
                        #print("diff= " + str(cv2.mean(diff)[0]))
                        cv2.imshow("face0", gray0)
                        cv2.imshow("face1", gray1)
                        cv2.rectangle(frame_out, (x0, y0), (x0 + w0, y0 + h0), (0, 0, 255), 2)
                        cv2.rectangle(frame_out, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 255), 2)    
    
    
    ####    
    #將找到的臉已藍框框起來
    #將找到的路人全部用綠色框起來
    for (x, y, w, h) in locations:
        cv2.rectangle(frame_out, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    
        
    cv2.imshow("find_two_look_alike", frame_out)
    
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

找路人只需很簡單的使用cv2.HOGDescriptor()的方法  
找臉也只需要只用face_cascade.detectMultiScale()  
麻煩的是找兩個相似的臉  
如果使用cv2.matchTemplate可以將所有的臉都當成一次模板與其他臉都做一次比對  
但光是像第1題一樣只使用一個模板去偵測的速度就已經很慢了  
何況這邊要使用所有的臉都當一次模板      
由於兩個臉大小不一樣故需要先做resize統一大小  
接著用cv2.absdiff()讓每個臉對各自進行比對  
當相異程度小於一定的值，則表示他們是一樣的臉  
另一個麻煩的是設定face_cascade.detectMultiScale()的值  
設定太小會找到很多不是臉的東西，設定太大則很難找到臉  
為了多找幾個臉，這邊偏好讓他設定小一點，找錯也沒關係，目標是找到兩個相似的臉  

In [7]:
#game_3 : "find_the_fastest_character"

import cv2
import numpy as np

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
frame_seq = 2480
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
status_cap, frame0 = cap.read()
#
#找路人
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
height, width =frame.shape[0:2]
locations, weights = hog.detectMultiScale(frame, hitThreshold = 0.15, winStride=(1,1), groupThreshold = 1)


#
#抓到第一偵的人臉
#face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.03, minNeighbors=3)
#建立多tracker清單
trackers = cv2.legacy.MultiTracker_create()
#初始化tracker清單
'''
if len(locations) > 0:
    x, y, w, h = locations[0]
    print("----")
    print(x, y, w, h)
    bbox = (x, y, w, h)
    tracker = cv2.legacy.TrackerMIL_create()
    status_tracker = tracker.init(frame, bbox)
else:
    print("err")
    exit()
    
'''
#選擇初始區域
bounding_boxes = []
for (x,y,w,h) in locations:
    bbox = (x, y, w, h)
    #將抓到的每一個人臉個別建立tracker並加入多tracker清單之中
    bounding_boxes.append(bbox)
    tracker = cv2.legacy.TrackerMIL_create()
    trackers.add(tracker,frame, bbox)

#先在第一偵以藍色框畫出之後將要tracker的目標
first_frame = np.copy(frame0)
for i, new_box in enumerate(bounding_boxes):
            x, y, w, h = [int(v) for v in new_box]
            cv2.rectangle(first_frame, (x, y), (x+w, y+h), (255, 0, 0), 3)
cv2.imshow("first frame", first_frame)
while True:
    frame_seq += 1
    if frame_seq > 2600:
        frame_seq = 2480
    cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
    status_cap, frame0 = cap.read()
    if not status_cap:
        break
    frame = cv2.resize(frame0, None, fx=scaling_factor, fy=scaling_factor)
    ###
    #for (x,y,w,h) in face_rects:
    #    print("****")
    #    print(x,y,w,h)
    #print("----")
    #face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=3)
    
    
    #持續刷新tracker 
    status_tracker, bboxes = trackers.update(frame)
    max_displacement = 0
    max_index = -1
    if status_tracker:
        for i, new_box in enumerate(bboxes):
            x, y, w, h = [int(v) for v in new_box]
            #將每個抓到的臉都畫出綠色框
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
            
            #計算每一個框的位移量，找出最大，紀錄此框的index
            displacement = (x - bounding_boxes[i][0]) ** 2 + (y - bounding_boxes[i][1]) ** 2
            if displacement > max_displacement:
                max_displacement = displacement
                max_index = i
    
    #將找到的最大框的index
    print("bboxes[max_index] = ")
    print(bboxes[max_index])
    x, y, w, h = bboxes[max_index]
    print("x, y, w, h")
    print(x, y, w, h)
    #以紅色框畫出位移量最大的物件
    cv2.rectangle(frame, (int(x), int(y)), (int(x)+int(w), int(y)+int(h)), (0,0 , 255), 3)
    
    #for (x,y,w,h) in face_rects:
    #    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
    
    ###
    cv2.imshow("find_the_fastest_character", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

bboxes[max_index] = 
[500.  91.  84. 168.]
x, y, w, h
500.0 91.0 84.0 168.0
bboxes[max_index] = 
[499.  92.  84. 168.]
x, y, w, h
499.0 92.0 84.0 168.0
bboxes[max_index] = 
[501.  91.  84. 168.]
x, y, w, h
501.0 91.0 84.0 168.0
bboxes[max_index] = 
[503.  92.  84. 168.]
x, y, w, h
503.0 92.0 84.0 168.0
bboxes[max_index] = 
[332.   0.  65. 130.]
x, y, w, h
332.0 0.0 65.0 130.0
bboxes[max_index] = 
[1128.  302.  113.  226.]
x, y, w, h
1128.0 302.0 113.0 226.0
bboxes[max_index] = 
[1132.  303.  113.  226.]
x, y, w, h
1132.0 303.0 113.0 226.0
bboxes[max_index] = 
[1135.  306.  113.  226.]
x, y, w, h
1135.0 306.0 113.0 226.0
bboxes[max_index] = 
[1135.  308.  113.  226.]
x, y, w, h
1135.0 308.0 113.0 226.0
bboxes[max_index] = 
[1139.  314.  113.  226.]
x, y, w, h
1139.0 314.0 113.0 226.0


原本想說偵測行人的方法速度比較慢，享用人臉來追蹤的，但後來發現找行人的效果比較好  
用cv2.legacy.MultiTracker_create()來建立多個Tracker的清單  
用trackers.add(tracker,frame, bbox)將偵測到個每一個locations加入tracker  
a)先在第一幀依照locations劃出藍色矩形 
用trackers.update(frame)來持續追蹤tracker到的臉  
b)接著用trackers.update(frame)回傳的bboxes畫出綠色矩形，同時計算哪個位移量最高  
c)因為在b已經計算出bboxes上第幾個index位移量快高，故只須根據這個index畫出紅色矩形  
同樣因為hog.detectMultiScale同個畫面每次抓地都不太一樣  
不一定每次都能抓到右上角那個在跑的人 所以可能要多嘗試幾次  

In [3]:
#game_4 : "find_two_odds"

import cv2
import numpy as np

def display_flow(img, flow, stride=10):
    for index in np.ndindex(flow[::stride, ::stride].shape[:2]):
        pt1 = tuple(i*stride for i in index)
        delta = flow[pt1].astype(np.int32)[::-1]
        pt2 = tuple(pt1 + 2*delta)
        if 2 <= cv2.norm(delta) <= 10:
            cv2.arrowedLine(img, pt1[::-1], pt2[::-1], (255,0,0), 1, cv2.LINE_AA, 0, 0.1)
    #norm_opt_flow = np.linalg.norm(flow, axis=2)
    #norm_opt_flow = cv2.normalize(norm_opt_flow, None, 0, 1, cv2.NORM_MINMAX)
    cv2.imshow('find_two_odds', img)
    #cv2.imshow('optical flow magnitude', norm_opt_flow)
    k = cv2.waitKey(1)
    if k == 27:
        return 1
    else:
        return 0

scaling_factor = 1
cap = cv2.VideoCapture("wiiplay.mp4")
ini_seq = 1680
frame_seq = ini_seq

cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)
_ , frame = cap.read()
frame0 = cv2.resize(frame, (0,0), None, 0.5, 0.5)
prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
prev_frame = cv2.resize(prev_frame, (0,0), None, 0.5, 0.5)
first_frame = True
####face_tracker

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


#抓到第一偵的人臉
face_rects = face_cascade.detectMultiScale(frame0, scaleFactor=1.0001, minNeighbors=3,maxSize = (50,50))
#建立多tracker清單
trackers = cv2.legacy.MultiTracker_create()
#初始化tracker清單
if len(face_rects) > 0:
    x, y, w, h = face_rects[0]
    print("----")
    print(x, y, w, h)
    bbox = (x, y, w, h)
    tracker = cv2.legacy.TrackerMIL_create()
    status_tracker = tracker.init(frame0, bbox)
else:
    print("err")
    exit()
#選擇初始區域
bounding_boxes = []

for (x,y,w,h) in face_rects:
    bbox = (x, y, w, h)
    #將抓到的每一個人臉個別建立tracker並加入多tracker清單之中
    bounding_boxes.append(bbox)
    tracker = cv2.legacy.TrackerMIL_create()
    trackers.add(tracker,frame0, bbox)

#先在第一偵以藍色框畫出之後將要tracker的目標
first_frame_ = np.copy(frame0)
for i, new_box in enumerate(bounding_boxes):
            x, y, w, h = [int(v) for v in new_box]
            cv2.rectangle(first_frame_, (x, y), (x+w, y+h), (255, 0, 0), 3)
cv2.imshow("first frame", first_frame_)

####
while True:
    frame_seq += 1
    if frame_seq > 1800:
        frame_seq = ini_seq
    status_cap, frame0 = cap.read()
    frame = np.copy(frame0)
    frame = cv2.resize(frame, (0,0), None, 0.5, 0.5)
    if not status_cap:
        break
    ################################
    #持續刷新tracker 
    b = 0
    r = 0
    
    color = []
    
    status_tracker, bboxes = trackers.update(frame)
    max_displacement = 0
    max_index = -1
    if status_tracker:
        for i, new_box in enumerate(bboxes):
            x, y, w, h = [int(v) for v in new_box]
            #將每個抓到的臉都畫出綠色框
            #cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
            
            #計算每一個框的位移量，找出最大，紀錄此框的index
            displacement = (x - bounding_boxes[i][0])
            if displacement > 4 :#往右位移藍色框
                #print("blue"+str(displacement))
                #cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0, 0), 3)
                color.append(1)
                b+=1
            elif displacement < -4 :#往左位移紅色框
                #print("red"+str(displacement))
                #cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 3)
                
                color.append(-1)
                r+=1
            else :
                color.append(0)
                #print("greeb"+str(displacement))
                #cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255, 0), 3)

    
    if b>r:
        for i in range(len(color)):
            if color[i] ==-1:
                x, y, w, h = bboxes[i]
                cv2.rectangle(frame, (int(x), int(y)), (int(x)+int(w), int(y)+int(h)), (0,0 ,255), 3)
    elif b<r :
        for i in range(len(color)):
            if color[i] ==1:
                x, y, w, h = bboxes[i]
                cv2.rectangle(frame, (int(x), int(y)), (int(x)+int(w), int(y)+int(h)), (0,0 ,255), 3)
    color.clear()
    #######################################
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #use cv2.calcOpticalFlowPyrLK() for sparse optical flow
    if first_frame:
        opt_flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, None, 0.5, 5, 13, 10, 5, 1.1, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        first_frame = False
    else:
        opt_flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, opt_flow, 0.5, 5, 13, 10, 5, 1.1, cv2.OPTFLOW_USE_INITIAL_FLOW)
    prev_frame = np.copy(gray)
    if display_flow(frame, opt_flow):
        break;
        
cap.release()
cv2.destroyAllWindows()

----
312 84 30 30


嘗試使用mp.solutions.face_detection找人臉，但不適用於wii的小人當中  
也有想過用光流的方式找出頭往左邊側與右邊側的人，但移動量太小，光流基本上無法判斷  
最後使用tracker的方式，第一偵先跳過第幾關的畫面，直接跳到關卡開始的時候，以這偵的圖像找人臉  
因為需要盡可能地多抓到幾個，所以導致參數設得很低，計算量變多，偵數下降  
但透過找到大部分的臉以後，計算每個臉的x軸位移量判斷他們正在往左看還是往右看，並統計兩者數量  
因為只有兩個人是往不同方向看，所以就可以判斷是哪兩個人與其他人看的方向不一樣  
最後就可以將這兩個人以紅色框起來  
雖然有時候會抓到奇怪的框/臉，但在轉向時基本上都能抓出來。  

In [2]:
#game_5: hand gestures of Rock, Scissor, Paper
#pip install mediapipe
import cv2
import mediapipe as mp

import math

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
############################
#計算兩點的角度
def vector_2d_angle(v1, v2):
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    #print(v1_x ,v1_y,v2_x ,v2_y)
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 180
    return angle_

#根據傳來的手計算各個需要的角度
def hand_angle(hand_):
    angle_list = []
    # thumb 大拇指
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    # index 食指
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    # middle 中指
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    # ring 無名指
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    # pink 小拇指
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list
#根據各個手指的角度，判斷目前是哪個手勢
def hand_pos(finger_angle):
    f1 = finger_angle[0]   # 大拇指
    f2 = finger_angle[1]   # 食指
    f3 = finger_angle[2]   # 中指
    f4 = finger_angle[3]   # 無名指
    f5 = finger_angle[4]   # 小拇指
    #print(f1,f2,f3,f4,f5)
    if f1>=50 and f2>=50 and f3>=50 and f4>=50 and f5>=50:
        return 'stone'
    elif f1>=50 and f2<50 and f3<50 and f4>=50 and f5>=50:
        return 'scissors'
    elif f1<50 and f2<50 and f3<50 and f4<50 and f5<50:
        return 'paper'
    else:
        return ''


################################
# For webcam input:
cap = cv2.VideoCapture("HG.mp4")
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
w = frameWidth
h = frameHeight

with mp_hands.Hands(model_complexity=0,min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        results = hands.process(img2)               

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                finger_points = []                   # 記錄手指的點
                for i in hand_landmarks.landmark:
                    #將這些點轉換成圖上的座標
                    x = i.x*w
                    y = i.y*h
                    finger_points.append((x,y))
                    cv2.circle(image,(int(x),int(y)), 10, (255, 0, 0), 2)
                if finger_points:
                    finger_angle = hand_angle(finger_points) # 計算手指角度
                    text = hand_pos(finger_angle)            # 取得手勢所回傳的內容
                    cv2.putText(image, text, (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 3, cv2.LINE_AA) # 印出文字
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
        cv2.imshow('MediaPipe Hands', image)
        c = cv2.waitKey(1)
        if c == 27:
            break
cap.release()
cv2.destroyAllWindows()

程式碼參考 https://steam.oxxostudio.tw/category/python/ai/ai-mediapipe-gesture.html  
他把0~9的手勢都做出來，我看了一下把需要的手勢拿出來改一下  
"2" -> 剪刀 "0" -> 石頭  "5" -> 布  
主要的方法就是計算五隻手指與掌心的角度  
到了一定角度代表這隻手指是捲曲狀態  
剪刀 -> 除了食指中指以外都捲曲  
石頭 -> 全部捲曲  
布 -> 都沒捲曲  
經過測試，旋轉、縮放應該可以一樣能判斷出來  

###### %%writefile test
## Final Exam
<b>The general objective is to solve five different games.<br>
The code for each game should be written in an individual cell.<br>
Add comments in your code to explain your approach.<br>
You can design your own approach, and use any method you learned in this class.</b><br>

1A. Input images from video file WiiPlay.mp4 with level 15 (frame number between 4820 and 5000).<br> 
1B. (5pts) Acquire a <b>face template</b> from the first frame (frame number = 4820).<br>
1C. (10pts) Try to detect the face the same as the template on subsequent frames, draw a <b>red</b> rectangle around the detected face, and show the output images in the <b>"find_this_mii"</b> window.<br><br>

2A. Input images from video file WiiPlay.mp4 with level 8 (frame number between 2180 and 2380).<br>
2B. (5pts) Detect <b>pedestrians</b> on each frame and draw a <b>green</b> rectangle around your detection.<br>
2C. (5pts) Detect <b>faces</b> on each frame and draw a <b>blue</b> rectangle around your detection.<br>
2D. (10pts) Try to find two faces look like each other, draw a <b>red</b> rectangle around each of the two faces, and show the output images in the <b>"find_two_look_alike"</b> window.<br><br>

3A. Input images from video file WiiPlay.mp4 with level 9 (frame number between 2480 and 2600).<br>
3B. (5pts) <b>Detect </b>faces(or pedestrians) on the first frame and draw a <b>blue</b> rectangle around your detection.<br>
3C. (10pts) <b>Track </b>faces(or pedestrians) on subsequent frames and draw a <b>green</b> rectangle around your tracking.<br>
3D. (5pts) Try to find out the fastest character, draw a <b>red</b> rectangle around the fastest character, and show the output images in the <b>"find_the_fastest_character"</b> window.<br><br>

4A. Input images from video file WiiPlay.mp4 with level 6 (frame number between 1650 and 1800).<br>
4B. (10pts) Compute and show <b>optical flows</b> on each frame using <b>blue</b> arrows.<br>
4C. (5pts) Try to detect two odd character who face the opposite direction from everyone else, draw a <b>red</b> rectangle around each of the two character, and show the output images in the <b>"find_two_odds"</b> window.<br><br>

5A. Input continuous BGR images from webcam.<br>
5B. (5pts) Use <i>MediaPipe()</i> to detect and track one of your hands.<br>
5C. (5pts) Obtain the positions of 21 <b>HandLandmarks</b>, draw a <b>blue</b> circle around each HandLandmark.<br>
5D. (10pts) Design an algorithm to recognize three hand gestures of <b>Rock, Scissor, Paper</b>. Write the type of the recognized hand gesture on the upper left corner using <i>cv2.putText()</i>.<br>
5E. In additon to translation, can your method correctly handle <b>rotated</b> (5pts bonus) and <b>scaled</b> (5pts bonus) hand gestures?<br><br>

6. (5pts) Any comments regarding the final exam? Which steps you believe you have completed? Which steps bother you?<br>
7. (5pts) Any suggestion to teaching assistants to improve this class? Any suggestion to teacher to improve this class?<br>
8. Upload your Jupyter file (*.ipynb) with code (for 1-5) and report (for 6-7). 

## Report
6.沒甚麼意見。全部的步驟基本上都有找到方法解決，但有些不太確定是自身硬體太爛的關係所以成效不佳，速度很慢。  
只要是用到face_cascade.detectMultiScale的方式都蠻卡的，Template也很卡  
主要最麻煩困擾的一題應該是第4題，原本是想說去"判斷"每個人的臉是往左擺還是往右擺  
想到的辦法只有1.自行訓練資料 然後再去去測 2.以上課給的人臉訓練資料，將計算每個臉的眼睛、嘴等等角度去判斷  
但1.有點難，而且這樣還要另外上傳訓練後的model，2.則是在第4題有提到，該訓練資料不能辨識wii的小人臉  
後來想說能不能用光流的方式去算各個臉是正要往左還是往右擺，但影片的幅度不夠，光流基本上沒有  
最後想到的方法就只有用trackr計算抓到的每個臉的移動軌跡，將正在往左擺與正在往右擺做分類  
最少的那個即是我們要的。  

7.無。硬要說的話就是好不容易上課地點在VR教室，希望可以玩一玩VR。  
